# Final project - data retrieval

# DLS Final Project
Predicting school test scores based on landsat images

# Image collection using google api

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !pip install --force-reinstall geemap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.2/260.2 kB 24.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 

In [ ]:
import ee
import geemap
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import IPython.display as disp
from ipyleaflet import Map, basemaps, basemap_to_tiles
# import geemap.core as geemap
%matplotlib inline

## Need to authenticate the google earth engine credentials

In [ ]:
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=VQPugQq-XdcsGEFOd3jZmzEoXWfmRcbnA_AaIluGBh0&tc=bM6ICTJ2ljVulWF16urnLWcB48qg87hxSJhywnlGJCA&cc=Gz4dk4sIzaaJ4_LKm68vmVyIbE5C0QvwFQ0n5ccm9oQ

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlwwwtDujonWgfgxoISB1EB8rxNWI2SpNw6xVmIz33yihgCc4-jWAs

Successfully saved authorization token.


In [ ]:
ee.Initialize()

## School data

In [ ]:
school_data = pd.read_csv('/content/drive/MyDrive/final_project/US_school_name_location/Public_Schools.csv')
school_data = school_data[school_data['STATE']=='CA'].reset_index(drop=True)

In [ ]:
print(list(school_data.columns))

['X', 'Y', 'OBJECTID', 'NCESID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP', 'ZIP4', 'TELEPHONE', 'TYPE', 'STATUS', 'POPULATION', 'COUNTY', 'COUNTYFIPS', 'COUNTRY', 'LATITUDE', 'LONGITUDE', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCEDATE', 'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'LEVEL_', 'ENROLLMENT', 'ST_GRADE', 'END_GRADE', 'DISTRICTID', 'FT_TEACHER', 'SHELTER_ID']


In [ ]:
school_data.iloc[0]

X                                                -13510876.8757
Y                                                  4665941.5775
OBJECTID                                                     10
NCESID                                              60143213940
NAME                               PARAMOUNT COLLEGIATE ACADEMY
ADDRESS                                     4010 EL CAMINO AVE.
CITY                                                 SACRAMENTO
STATE                                                        CA
ZIP                                                       95821
ZIP4                                                       6602
TELEPHONE                                        (916) 484-1480
TYPE                                                          1
STATUS                                                        1
POPULATION                                                   77
COUNTY                                               SACRAMENTO
COUNTYFIPS                              

In [ ]:
lat = school_data['LATITUDE'].iloc[0]
lon = school_data['LONGITUDE'].iloc[0]

In [ ]:
print(lat)
print(lon)

38.609867949
-121.370271993


# Retrieving images using geemap and saving them as html files to open and take a screenshot of later

In [ ]:
school_dict = {}
for i in range(0,len(school_data.iloc[0:10])):
  lat = school_data['LATITUDE'].iloc[i]
  lon = school_data['LONGITUDE'].iloc[i]
  school_name = school_data['NAME'].iloc[i]
  school_dict[school_name] = {'latitude':lat, 'longitude':lon}
  i+=1

In [ ]:
for school in school_dict:
  lat = school_dict[school]['latitude']
  lon = school_dict[school]['longitude']
  bm = basemaps.Esri.WorldImagery
  ### Getting the smaller map
  m = geemap.Map(basemap=basemap_to_tiles(bm))
  m.set_center(lon, lat, 18)
  save_path = '/content/drive/MyDrive/final_project/sat_images/img_data_html_geemap/small/' + 'small__' + format(school) + '.html'
  m.save(save_path)
  ### Getting the larger map
  m = geemap.Map(basemap=basemap_to_tiles(bm))
  m.set_center(lon, lat, 15)
  save_path = '/content/drive/MyDrive/final_project/sat_images/img_data_html_geemap/large/' + 'large__' + format(school) + '.html'
  m.save(save_path)

# Testing of subsets of images below

In [ ]:
school_dict = {}
for i in range(0,len(school_data.iloc[0:1])):
  lat = school_data['LATITUDE'].iloc[i]
  lon = school_data['LONGITUDE'].iloc[i]
  school_name = school_data['NAME'].iloc[i]
  school_dict[school_name] = {'latitude':lat, 'longitude':lon}
  i+=1

In [ ]:
basemaps.Esri.WorldImagery()

In [ ]:
for school in school_dict:
  lat = school_dict[school]['latitude']
  lon = school_dict[school]['longitude']
  bm = basemaps.Esri.WorldImagery
  ### Getting the smaller map
  m = geemap.Map(basemap=basemap_to_tiles(bm), zoom_ctrl=False, fullscreen_ctrl=False, draw_ctrl=False, scale_ctrl=False, measure_ctrl=False,
                 search_ctrl=False, toolbar_ctrl=False)
  m.set_center(lon, lat, 18)
  m
  #save_path = '/content/' + 'small__' + format(school) + 'TEST.html'
  #m.save(save_path)

In [ ]:
m

Map(center=[38.609867949, -121.370271993], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

# Below is junkyard of failed attempts

# Attemp using google earth engine landsat sentinel images. Follows a tutorial from this link about comparing landsat images:
https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-1

**Benefits:**
- High resolution images

**Issues:**
- Seems to be only one zoom level that may also not be enough to see the school building itself.

In [ ]:
### Having differences of 0.05 on each side for a total of 0.1 degrees is 1.1km^2

lat_upper = lat + 0.05
lat_lower = lat - 0.05

lon_upper = lon + 0.05
lon_lower = lon - 0.05


### Tutorial script coords
lat_upper =  8.658599853515625
lat_lower = 8.473892211914062

lon_upper = 50.06066538593667
lon_lower = 49.98081240937428


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              lat_lower,
              lon_lower
            ],
            [
              lat_upper,
              lon_lower
            ],
            [
              lat_upper,
              lon_upper
            ],
            [
              lat_lower,
              lon_upper
            ],
            [
              lat_lower,
              lon_lower
            ]
          ]
        ]
      }
    }
  ]
}


In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


In [ ]:
# print(coords)
# print(aoi)

In [ ]:
### These are tier 1 landsat images, the highest quality
### Not specifying the orbit pass as descending or ascending will give different angles of the schools, which is great

ffa_db = ee.Image( ee.ImageCollection('LANDSAT/LC09/C02/T1')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))
                       .first()
                       .clip(aoi)
)

### data is available between these dates 2021-10-31T00:00:00Z–2023-11-30T02:29:53

In [ ]:
print(ffa_db)

ee.Image({
  "functionInvocationValue": {
    "functionName": "Image.clip",
    "arguments": {
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    8.473892211914062,
                    49.98081240937428
                  ],
                  [
                    8.658599853515625,
                    49.98081240937428
                  ],
                  [
                    8.658599853515625,
                    50.06066538593667
                  ],
                  [
                    8.473892211914062,
                    50.06066538593667
                  ],
                  [
                    8.473892211914062,
                    49.98081240937428
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constan

In [ ]:
ffa_db.getInfo()

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [444, 299],
   'origin': [2449, 4780],
   'crs': 'EPSG:32632',
   'crs_transform': [30, 0, 388785, 0, -30, 5688915]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [444, 299],
   'origin': [2449, 4780],
   'crs': 'EPSG:32632',
   'crs_transform': [30, 0, 388785, 0, -30, 5688915]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [444, 299],
   'origin': [2449, 4780],
   'crs': 'EPSG:32632',
   'crs_transform': [30, 0, 388785, 0, -30, 5688915]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [444, 299],
   'origin': [2449, 4780],
   'crs': 'EPSG:32632',
   'crs_transform': [30, 0, 388785, 0, -30, 5688

In [ ]:
ffa_db.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B9',
 'B10',
 'B11',
 'QA_PIXEL',
 'QA_RADSAT',
 'SAA',
 'SZA',
 'VAA',
 'VZA']

In [ ]:
### See the thumbnail image, which is weird because get individual image bands

url = ffa_db.select('B4').getThumbURL({'min': -20, 'max': 0})

#url = ffa_db.select('B1').getThumbURL()

disp.Image(url=url, width=800)


### Try to save the image using geemap

In [ ]:
ffa_db = ee.Image( ee.ImageCollection('LANDSAT/LC09/C02/T1')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))
                       .first()
                       .clip(aoi)
)

rgb = ee.Image.rgb(ffa_db.select('B4'),
                   ffa_db.select('B3'),
                   ffa_db.select('B2'))


In [ ]:
Map = geemap.Map()
Map.addLayer(rgb, {}, 'Landsat')
Map.centerObject(rgb)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
geemap.download_ee_image(rgb, 'test.tif', scale=100)

test.tif: |          | 0.00/73.2k (raw) [  0.0%] in 00:00 (eta:     ?)

## Trying slight variation of above but using composite

In [ ]:
### Having differences of 0.05 on each side for a total of 0.1 degrees is 1.1km^2

lat_upper = lat + 0.05
lat_lower = lat - 0.05

lon_upper = lon + 0.05
lon_lower = lon - 0.05


### Tutorial script coords
lat_upper =  8.658599853515625
lat_lower = 8.473892211914062

lon_upper = 50.06066538593667
lon_lower = 49.98081240937428


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              lat_lower,
              lon_lower
            ],
            [
              lat_upper,
              lon_lower
            ],
            [
              lat_upper,
              lon_upper
            ],
            [
              lat_lower,
              lon_upper
            ],
            [
              lat_lower,
              lon_lower
            ]
          ]
        ]
      }
    }
  ]
}


In [ ]:
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)


In [ ]:
ffa_db = ee.Image( ee.ImageCollection('LANDSAT/LC09/C02/T1')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))
)

In [ ]:
composite = ee.Algorithms.Landsat.simpleComposite(ffa_db).float();

In [ ]:
rgb = composite.select('B4','B3','B2').unitScale(0,255)
gray = composite.select('B8').unitScale(0,155)

In [ ]:
huesat = rgb.rgbToHsv().select('hue','saturation')
satellite_imagery = ee.Image.cat(huesat, gray).hsvToRgb()

In [ ]:
map = folium.Map(location = [lat,lon],
                 zoom_start=6,
                 )

NameError: ignored

### Function to download new images

In [ ]:
def image_download(lat, lon, school_name, scale):
  '''This function downloads images from Sentinal T1 images.
     Parameters:
     lat = latitude of school
     lon = longitude of school
     school_name = name of school
     scale = scale of the image. Lower scale means larger image. Ex: scale of 100 ~30mb, whereas scale of 10 ~3.3GB'''


  ### Having differences of 0.05 on each side for a total of 0.1 degrees is 1.1km^2
  lat_upper = lat + 0.15
  lat_lower = lat - 0.15

  lon_upper = lon + 0.15
  lon_lower = lon - 0.15

  ### Get an image from the Sentinel landsat images. T1 is the better quality. Also centers on the lat/long and within a date.
  ffa_db = ee.Image(ee.ImageCollection('LANDSAT/LC09/C02/T1')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))
                       .first())

  rgb = ee.Image.rgb(ffa_db.select('B4'),
                   ffa_db.select('B3'),
                   ffa_db.select('B2'))

  Map = geemap.Map()
  Map.addLayer(rgb, {}, 'Landsat')
  path = '/content/drive/MyDrive/final_project/sat_images/img_data_test/' + format(school_name) + '.tif'
  geemap.download_ee_image(rgb, path, scale=scale)


In [ ]:
idx = 1
lat = school_data.iloc[idx]['LATITUDE']
lon = school_data.iloc[idx]['LONGITUDE']
name = school_data.iloc[idx]['NAME']
print(lat)
print(lon)
print(name)

NameError: ignored

In [ ]:
image_download(lat, lon, name, scale = 50)

OLIVE GROVE CHARTER.tif: |          | 0.00/135M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
def get_data(idx):
  lat = school_data.iloc[idx]['LATITUDE']
  lon = school_data.iloc[idx]['LONGITUDE']
  name = school_data.iloc[idx]['NAME']
  print(lat)
  print(lon)
  print(name)
  return lat, lon, name

# Following youtube video:
https://www.youtube.com/watch?v=Fd1eYs49dUI
Result: did not work

In [ ]:
get_data(3969)

38.5292920030001
-121.439183585
WEST CAMPUS


(38.5292920030001, -121.439183585, 'WEST CAMPUS')

In [ ]:
school_data[school_data['NAME'] == 'WEST CAMPUS']

,X,Y,OBJECTID,NCESID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_METHOD,VAL_DATE,WEBSITE,LEVEL_,ENROLLMENT,ST_GRADE,END_GRADE,DISTRICTID,FT_TEACHER,SHELTER_ID
3969,-1.351855e+07,4.654469e+06,38334,63384010600,WEST CAMPUS,5022 58TH ST.,SACRAMENTO,CA,95820,5610,...,IMAGERY/OTHER,2010/08/30 00:00:00,NOT AVAILABLE,HIGH,862,09,12,633840,33,NOT AVAILABLE


In [ ]:
idx = 1
lat, lon, name = get_data(3969)

lat_upper = lat + 0.15
lat_lower = lat - 0.15

lon_upper = lon + 0.15
lon_lower = lon - 0.15

### Get an image from the Sentinel landsat images. T1 is the better quality. Also centers on the lat/long and within a date.
ffa_db = ee.Image(ee.ImageCollection('LANDSAT/LC09/C02/T1')
                      .filterBounds(aoi)
                      .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))
                      .first())

rgb = ee.Image.rgb(ffa_db.select('B4'),
                  ffa_db.select('B3'),
                  ffa_db.select('B2'))

Map = geemap.Map()
Map.addLayer(rgb, {}, 'Landsat')
Map.centerObject(rgb, 7)
Map

38.5292920030001
-121.439183585
WEST CAMPUS


Map(center=[50.2786614745982, 9.099173632896346], controls=(WidgetControl(options=['position', 'transparent_bg…

In [ ]:
idx = 1
lat, lon, name = get_data(3969)

lat_upper = lat + 0.15
lat_lower = lat - 0.15

lon_upper = lon + 0.15
lon_lower = lon - 0.15

countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
roi = countries.filter(ee.Filter.eq("country_na",'United States'))

### Get an image from the Sentinel landsat images. T1 is the better quality. Also centers on the lat/long and within a date.
landsat = ee.ImageCollection('LANDSAT/LC09/C02/T1')\
                      .filterBounds(aoi)\
                      .filterDate(ee.Date('2023-01-01'), ee.Date('2023-06-1'))

composite = ee.Algorithms.Landsat.simpleComposite(**{'collection':landsat, 'asFloat':True})

rgbVis = {'bands': ['B4','B3','B2'], 'min':0, 'max':0.3}

Map = geemap.Map()
Map.addLayer(roi, {}, "United States")
Map.addLayer(composite.clip(roi), rgbVis, "Landsat RGB")
Map

38.5292920030001
-121.439183585
WEST CAMPUS


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
### This is the tier 2 of the landsat images. Not as pretty, but contains more images

ffa_db = ee.Image( ee.ImageCollection('LANDSAT/LC09/C02/T2')
                       .filterBounds(aoi))


In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                       .first()
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                       .filterBounds(aoi)
                       .filterDate(ee.Date('2020-08-01'), ee.Date('2020-08-31'))
                       .first()
                       .clip(aoi))


# Geemap attempt
**Benefits:**
- Can zoom in and out on multiple levels to get differing zooms of schools.
- Easy to use for locations.

**Issues:**
- Provides an html file. Could try to save an html file, but do not know how to open them in a browser and take a screenshot and return back to the window using only python code.
- Cannot remove widgets from image, so they get in the way.

In [ ]:
### Masking map but also makes an interactive map that is annoying to scroll past


def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(lon, lat, 17)
#m.add_layer(dataset.mean(), visualization, 'RGB')
m


In [ ]:
### My house coords
lon = -76.681111
lat = 36.897222

lat=39.151111
lon=-86.481667

In [ ]:
m.basemap()

{'url': 'https://tile.openstreetmap.org/{z}/{x}/{y}.png',
 'max_zoom': 19,
 'html_attribution': '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
 'attribution': '(C) OpenStreetMap contributors',
 'name': 'OpenStreetMap.Mapnik'}

In [ ]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

In [ ]:
# "SATELLITE": "Esri.WorldImagery"
bm = basemaps.Esri.WorldImagery

In [ ]:
m = geemap.Map(basemap=basemap_to_tiles(bm))
m.set_center(lon, lat, 15)
m

Map(center=[38.609867949, -121.370271993], controls=(WidgetControl(options=['position', 'transparent_bg'], wid…

In [ ]:
m.save('test.html')

In [ ]:
geemap.map_widgets.Inspector

geemap.map_widgets.Inspector

In [ ]:
m.set_trait('ControlVisibility'=False)

SyntaxError: ignored

In [ ]:
m.get_draw_control

<bound method Map.get_draw_control of Map(bottom=3217917.0, center=[38.609867949, -121.370271993], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(children=(ToggleButton(value=False, icon='globe', layout=Layout(height='28px', padding='0px 0px 0px 4px', width='28px'), tooltip='Search location/data'),), _dom_classes=('geemap-colab',))), ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text', 'zoom_out_title']), FullScreenControl(options=['position']), MapDrawControl(marker={'shapeOptions': {'color': '#3388ff'}}, options=['position'], polygon={'shapeOptions': {}}, polyline={'shapeOptions': {}}, rectangle={'shapeOptions': {'color': '#3388ff'}}), ScaleControl(options=['imperial', 'max_width', 'metric', 'position', 'update_when_idle'], position='bottomleft'), MeasureControl(active_color='orange', options=['active_color', 'capture_z_index', 'completed_color', 'popup_options', 'position', 'primary_area_unit', 'primary_length_unit'

In [ ]:
lat = school_data['LATITUDE'].iloc[0]
lon = school_data['LONGITUDE'].iloc[0]

In [ ]:
sub = school_data.iloc[0:10]

In [ ]:
school_dict = {}
for i in range(0,len(school_data.iloc[0:10])):
  lat = school_data['LATITUDE'].iloc[i]
  lon = school_data['LONGITUDE'].iloc[i]
  school_name = school_data['NAME'].iloc[i]
  school_dict[school_name] = {'latitude':lat, 'longitude':lon}
  i+=1

In [ ]:
school_dict

{'PARAMOUNT COLLEGIATE ACADEMY': {'latitude': 38.609867949,
  'longitude': -121.370271993},
 'OLIVE GROVE CHARTER': {'latitude': 34.943665142,
  'longitude': -119.674680762},
 'THE ACADEMY OF ALAMEDA ELEMENTARY': {'latitude': 37.777044802,
  'longitude': -122.285420549},
 'JIM?NEZ ROBERTO AND DR. FRANCISCO ELEMENTARY SCHOOL': {'latitude': 34.9251573070001,
  'longitude': -120.450339288},
 'BURBANK UNIFIED INDEPENDENT LEARNING ACADEMY (BUILA)': {'latitude': 34.178349682,
  'longitude': -118.351135217},
 'CASTLEMONT PRIMARY ACADEMY': {'latitude': 37.7577522650001,
  'longitude': -122.163031457},
 'FRANCOPHONE CHARTER SCHOOL OF OAKLAND': {'latitude': 37.751434848,
  'longitude': -122.157021795},
 'INNOVATIONS ACADEMY OF PALMDALE': {'latitude': 34.5757269540001,
  'longitude': -118.066504984},
 'OPEN VALLEY INDEPENDENT STUDY': {'latitude': 37.47573276,
  'longitude': -121.140319052},
 'PINNACLES COMMUNITY': {'latitude': 36.8545639430001,
  'longitude': -121.396908701}}

In [ ]:
for school in school_dict:
  lat = school_dict[school]['latitude']
  lon = school_dict[school]['longitude']
  bm = basemaps.Esri.WorldImagery
  ### Getting the smaller map
  m = geemap.Map(basemap=basemap_to_tiles(bm), ZoomControl = ))
  m.set_center(lon, lat, 18)
  save_path = '/content/img_data/small/' + 'small__' + format(school) + '.html'
  m.save(save_path)
  ### Getting the larger map
  m = geemap.Map(basemap=basemap_to_tiles(bm))
  m.set_center(lon, lat, 15)
  save_path = '/content/img_data/large/' + 'large__' + format(school) + '.html'
  m.save(save_path)

### Now trying to retrieve the html
PROBLEMS:
- The driver needs to be installed, but does not have one for the current version of chrome and it no longer works for firefox

In [ ]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
html = open("/content/drive/MyDrive/final_project/sat_images/img_data_html_geemap/small/small__BURBANK UNIFIED INDEPENDENT LEARNING ACADEMY (BUILA).html", 'r')
html_full = html.read()

In [ ]:
html_full

'<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <title>IPyWidget export</title>\n</head>\n<body>\n\n\n<!-- Load require.js. Delete this if your page already loads require.js -->\n<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script>\n<script src="https://unpkg.com/@jupyter-widgets/html-manager@^0.20.1/dist/embed-amd.js" crossorigin="anonymous"></script>\n\n<script type="application/vnd.jupyter.widget-state+json">\n{\n  "version_major": 2,\n  "version_minor": 0,\n  "state": {\n    "c6fce7ff0c404674a8e509fe6a936dd0": {\n      "model_name": "LeafletTileLayerModel",\n      "model_module": "jupyter-leaflet",\n      "model_module_version": "^0.17",\n      "state": {\n        "attribution": "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community",\n    

In [ ]:
DRIVER = 'chromedriver'
driver = webdriver.Chrome(DRIVER)
driver.get()

In [ ]:

driver = webdriver.Chrome()
driver.get("https://www.ebay.com/sch/i.html?_from=R40&_trksid=p2334524.m570.l1313&_nkw=laptop&_sacat=0&LH_TitleDesc=0&_osacat=0&_odkw=laptop")
results = []
content = driver.page_source
soup = BeautifulSoup(content)


def parse_image_urls(classes, location, source):
    for a in soup.findAll(attrs={"class": classes}):
        name = a.find(location)
        if name not in results:
            results.append(name.get(source))


parse_image_urls("s-item__image-wrapper image-treatment", "img", "src")

SessionNotCreatedException: ignored

### Random testing

In [ ]:
landsat.getInfo(test_point).get_info()

TypeError: ignored

In [ ]:
centroi = test_point.centroid()

In [ ]:
from IPython.display import Image

In [ ]:
coll = landsat.fromImages([ee.Image(0), ee.Image(1)])

In [ ]:
landsat

# Attempting a different tutorial
https://carpentries-incubator.github.io/geospatial-python/05-access-data.html

### Results
- Can't get the true color image easily
- Do not see a way to determine resolution and thherefore it is super high up. Can't get closer photos

In [ ]:
from pystac_client import Client
from shapely.geometry import Point
import rioxarray

In [ ]:
client = Client.open("https://earth-search.aws.element84.com/v1")

In [ ]:
collection = "sentinel-2-l2a"

In [ ]:
point = Point(lon, lat)

### This is the tutorial coords
#point = Point(4.89, 52.37)

### White house coords
point = Point(-77.036560, 38.897957)

### My houes coords
point = Point(-76.681111, 36.897222)
bbox = point.buffer(0.05).bounds

In [ ]:
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
    query=["eo:cloud_cover<10"]
)

In [ ]:
print(search.matched())

155


In [ ]:
items = search.item_collection()

In [ ]:
print(len(items))

10


In [ ]:
for item in items:
  print(item)

<Item id=S2B_18SUF_20231116_0_L2A>
<Item id=S2A_18SUF_20231022_0_L2A>
<Item id=S2B_18SUF_20231017_0_L2A>
<Item id=S2A_18SUF_20231002_0_L2A>
<Item id=S2A_18SUF_20230902_0_L2A>
<Item id=S2B_18SUF_20230818_0_L2A>
<Item id=S2A_18SUF_20230813_0_L2A>
<Item id=S2A_18SUF_20230614_0_L2A>
<Item id=S2B_18SUF_20230609_0_L2A>
<Item id=S2B_18SUF_20230410_0_L2A>


In [ ]:
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

2023-11-16 16:03:06.180000+00:00
{'type': 'Polygon', 'coordinates': [[[-77.2483684043295, 37.025266990733755], [-77.21990635255229, 36.03617691222183], [-76.00146317821306, 36.052549930385496], [-76.01431366197991, 37.042238132699175], [-77.2483684043295, 37.025266990733755]]]}
{'created': '2023-11-16T20:42:53.263Z', 'platform': 'sentinel-2b', 'constellation': 'sentinel-2', 'instruments': ['msi'], 'eo:cloud_cover': 0.004088, 'proj:epsg': 32618, 'mgrs:utm_zone': 18, 'mgrs:latitude_band': 'S', 'mgrs:grid_square': 'UF', 'grid:code': 'MGRS-18SUF', 'view:sun_azimuth': 166.311435091446, 'view:sun_elevation': 33.521741172279, 's2:degraded_msi_data_percentage': 0, 's2:nodata_pixel_percentage': 3e-06, 's2:saturated_defective_pixel_percentage': 0, 's2:dark_features_percentage': 0, 's2:cloud_shadow_percentage': 0, 's2:vegetation_percentage': 59.396386, 's2:not_vegetated_percentage': 29.860422, 's2:water_percentage': 10.738067, 's2:unclassified_percentage': 0.000783, 's2:medium_proba_clouds_percen

## Accesing the images from the downloaded metadata

In [ ]:
assets = items[0].assets
print(assets.keys())

dict_keys(['aot', 'blue', 'coastal', 'granule_metadata', 'green', 'nir', 'nir08', 'nir09', 'red', 'rededge1', 'rededge2', 'rededge3', 'scl', 'swir16', 'swir22', 'thumbnail', 'tileinfo_metadata', 'visual', 'wvp', 'aot-jp2', 'blue-jp2', 'coastal-jp2', 'green-jp2', 'nir-jp2', 'nir08-jp2', 'nir09-jp2', 'red-jp2', 'rededge1-jp2', 'rededge2-jp2', 'rededge3-jp2', 'scl-jp2', 'swir16-jp2', 'swir22-jp2', 'visual-jp2', 'wvp-jp2'])


In [ ]:
for key, asset in assets.items():
  print(f"{key}: {asset.title}")

aot: Aerosol optical thickness (AOT)
blue: Blue (band 2) - 10m
coastal: Coastal aerosol (band 1) - 60m
granule_metadata: None
green: Green (band 3) - 10m
nir: NIR 1 (band 8) - 10m
nir08: NIR 2 (band 8A) - 20m
nir09: NIR 3 (band 9) - 60m
red: Red (band 4) - 10m
rededge1: Red edge 1 (band 5) - 20m
rededge2: Red edge 2 (band 6) - 20m
rededge3: Red edge 3 (band 7) - 20m
scl: Scene classification map (SCL)
swir16: SWIR 1 (band 11) - 20m
swir22: SWIR 2 (band 12) - 20m
thumbnail: Thumbnail image
tileinfo_metadata: None
visual: True color image
wvp: Water vapour (WVP)
aot-jp2: Aerosol optical thickness (AOT)
blue-jp2: Blue (band 2) - 10m
coastal-jp2: Coastal aerosol (band 1) - 60m
green-jp2: Green (band 3) - 10m
nir-jp2: NIR 1 (band 8) - 10m
nir08-jp2: NIR 2 (band 8A) - 20m
nir09-jp2: NIR 3 (band 9) - 60m
red-jp2: Red (band 4) - 10m
rededge1-jp2: Red edge 1 (band 5) - 20m
rededge2-jp2: Red edge 2 (band 6) - 20m
rededge3-jp2: Red edge 3 (band 7) - 20m
scl-jp2: Scene classification map (SCL)
swi

In [ ]:
band_key_list = ['coastal','blue','green','red','rededge1','rededge2','rededge3','nir','nir09','swir16','swir22']

In [ ]:
print(assets['thumbnail'].href)

https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/18/S/UF/2023/11/S2B_18SUF_20231116_0_L2A/thumbnail.jpg


In [ ]:
nir_href = assets['nir'].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)

<xarray.DataArray (band: 1, y: 10980, x: 10980)>
[120560400 values with dtype=uint16]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 6e+05 6e+05 6e+05 ... 7.098e+05 7.098e+05 7.098e+05
  * y            (y) float64 4.3e+06 4.3e+06 4.3e+06 ... 4.19e+06 4.19e+06
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:       Area
    OVR_RESAMPLING_ALG:  AVERAGE
    _FillValue:          0
    scale_factor:        1.0
    add_offset:          0.0


In [ ]:
### Saves whole image, which is ~150mb
# nir.rio.to_raster("img_data/test.tif")

### Saves portion of the image, which is about 1mb
nir[0, 1500:2200, 1500:2200].rio.to_raster("img_data/test_subset.tif")

In [ ]:
### Saving each band
for key in band_key_list:
  key_href = assets[key].href
  href = rioxarray.open_rasterio(key_href)
  #save_path = 'img_data/test_subset_' + str(key) + '.tif' ### Save path to google colab
  save_path = '/content/drive/MyDrive/final_project/sat_images/' + str(key) + '.tif' ### Save path to google drive
  href[0, 1500:2200, 1500:2200].rio.to_raster(save_path)

In [ ]:
key_href = assets['visual-jp2'].href
print(key_href)
href = rioxarray.open_rasterio(key_href)

s3://sentinel-s2-l2a/tiles/10/S/FH/2023/11/25/0/TCI.jp2


RasterioIOError: ignored

In [ ]:
key_href = assets['visual.jp2'].href
print(key_href)
href = rioxarray.open_rasterio(key_href)

KeyError: ignored

In [ ]:
from google.colab import files
import glob

In [ ]:
#glob.glob('/content/img_data/*')